# Plotting Financial Data with Plotly: Linear Regressions, 20 day moving averages, Bollinger Bands for Technical Analysis

### We are using Google Colaboratory.  Think of it as the Google Docs version of Jupyter notebook.  These 'Colaboratory' notebooks come preinstalled with all of the necessary Python code including GPU configurations.  If you need other packages, they are a 'pip' or 'conda' install away

In [3]:
! pip install plotly

#### Since colaboratory is hosted on a different computer but is a plotting tool, we just have to add some code to get it to work in google Colaboratory so that it renders on our screen

In [0]:
import warnings
warnings.filterwarnings("ignore")

#https://colab.research.google.com/notebook#fileId=14oudHx5e5r7hm1QcbZ24FVHXgVPD0k8f
import IPython
def configure_plotly_browser_state():
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [5]:
IPython.get_ipython().events.unregister('pre_run_cell', configure_plotly_browser_state)



ValueError: ignored

### Let's pull Historical data on Microsoft using the Quandl API

In [6]:
import requests

api_key = 'uqsmdRnZoXuRF3LRev__'
parameters = {'api_key':api_key}

initial_url = 'https://www.quandl.com/api/v3/datasets/'
dataset_group = 'EOD/'
dataset = 'MSFT'
datatype = '.json'

final_url = initial_url + dataset_group + dataset + datatype

results = requests.get(final_url, params=parameters)

APIresults = results.json()

APIresults.keys()

[u'dataset']

In [7]:
APIresults['dataset'].keys()

[u'dataset_code',
 u'column_names',
 u'newest_available_date',
 u'description',
 u'end_date',
 u'data',
 u'start_date',
 u'limit',
 u'transform',
 u'order',
 u'refreshed_at',
 u'frequency',
 u'database_code',
 u'oldest_available_date',
 u'collapse',
 u'premium',
 u'database_id',
 u'column_index',
 u'type',
 u'id',
 u'name']

#### So the data we want exists as a list of lists in the key '**data**' and the column names of that data exist in '**columns_names**' attribute.

In [8]:
APIresults['dataset']['column_names']

[u'Date',
 u'Open',
 u'High',
 u'Low',
 u'Close',
 u'Volume',
 u'Dividend',
 u'Split',
 u'Adj_Open',
 u'Adj_High',
 u'Adj_Low',
 u'Adj_Close',
 u'Adj_Volume']

In [9]:
APIresults['dataset']['data'][:2]

[[u'2018-11-16',
  107.08,
  108.88,
  106.8,
  108.29,
  33502121.0,
  0.0,
  1.0,
  107.08,
  108.88,
  106.8,
  108.29,
  33502121.0],
 [u'2018-11-15',
  104.99,
  107.8,
  103.91,
  107.28,
  38505165.0,
  0.0,
  1.0,
  104.99,
  107.8,
  103.91,
  107.28,
  38505165.0]]

### Convert the results into a Pandas DataFrame

In [10]:
import pandas as pd

msft = pd.DataFrame(data = APIresults['dataset']['data'], 
                    columns = APIresults['dataset']['column_names'])

msft.head(2)

,Date,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
0,2018-11-16,107.08,108.88,106.80,108.29,33502121.0,0.0,1.0,107.08,108.88,106.80,108.29,33502121.0
1,2018-11-15,104.99,107.80,103.91,107.28,38505165.0,0.0,1.0,104.99,107.80,103.91,107.28,38505165.0


### Sort the dataframe so that the earliest date comes first

In [11]:
print(type(msft['Date'][0]))

msft['Date'] = pd.to_datetime(msft['Date'])

print(type(msft['Date'][0]))

msft.set_index('Date',inplace=True)

msft.sort_index(inplace=True)

<type 'unicode'>
<class 'pandas._libs.tslib.Timestamp'>


In [12]:
msft.head(2)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
1986-03-13,25.5,29.25,25.5,28.0,3582600.0,0.0,1.0,0.057937,0.066457,0.057937,0.063617,1.031789e+09
1986-03-14,28.0,29.50,28.0,29.0,1070000.0,0.0,1.0,0.063617,0.067025,0.063617,0.065889,3.081600e+08


### Filter for years between 2010 and 2017

In [13]:
print(len(msft))

msft2 = msft[ (msft.index >= "2010-01-01") & (msft.index <= "2017-01-01") ]

print(len(msft2))

8241
1762


### We will be using a graphing package called Plotly in order to create an OHLC (Open High Low Close) graph for the Microsoft Stock Data


In [14]:
from plotly.offline import download_plotlyjs, init_notebook_mode,iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

### First we create the chart data

In [19]:

ohlc_line = go.Ohlc(x = msft2.index,
                       open = msft2['Open'],
                       high = msft2['High'],
                       low = msft2['Low'],
                       close = msft2['Close'],
                       name = 'OHLC')

# insert the ohlc_line into a list, per the plotly functionality

data = [ ohlc_line ]

# plot the data
configure_plotly_browser_state()
iplot(data)

### Eh, scrolling and zooming and dynamic changes are cool, but I want to add some extra parts to this graph....including:

- a line showing when Satya Nadella became CEO
- titles for graph, x-axis, y-axis
- A line showing the Close values

In [20]:
# "Layout" contains the extra visuals in the graph

layout = go.Layout({
    'title': "Microsoft Historical Stock Data",
    'yaxis': {'title': 'MSFT Stock ($)'},
    'xaxis' : {'rangeslider':{'visible':False}},
    'shapes': [{
        'x0': '2014-02-04', 'x1': '2014-02-04',
        'y0' : 0, 'y1': 1, 'xref': 'x', 'yref':'paper',
        'line' : {'color': 'rgb(30,30,30)', 'width':1}
    }],
    'annotations': [{
        'x': '2014-02-04', 'y':0.05, 'xref': 'x', 'yref': 'paper',
        'showarrow':False, 'xanchor': 'left',
        'text': 'Satya Nadella becomes CEO'
    }]
})

#### now lets create the line chart connecting all the close values together with a black line

close_line = go.Scatter(
    x=msft2.index,
    y = msft2['Close'],
    name= 'Closing $',
    line = {'color' : 'black','width' : 0.3 }
)

# create a data list containing all the charts we want to graph
data = [ ohlc_line, close_line ]

#  combine them with the layouts into a figure

fig = go.Figure(data=data, layout = layout)

# plot the data
configure_plotly_browser_state()
iplot(fig)

### So that was fun, let's drill down and analyze the past 2 years of data

In [21]:
msft3 = msft[msft.index >= "2017-01-01"]
msft3.tail(3)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2018-11-14,108.10,108.26,104.47,104.97,39495141.0,0.46,1.0,108.10,108.26,104.47,104.97,39495141.0
2018-11-15,104.99,107.80,103.91,107.28,38505165.0,0.00,1.0,104.99,107.80,103.91,107.28,38505165.0
2018-11-16,107.08,108.88,106.80,108.29,33502121.0,0.00,1.0,107.08,108.88,106.80,108.29,33502121.0


### Graph the close of msft3

In [23]:
close_line = go.Scatter(
    x=msft3.index,
    y = msft3['Close'],
    name= 'Closing $',
    line = {'color' : 'black','width' : 1 }
)

data = [close_line ]

# plot the data
configure_plotly_browser_state()
iplot(data)

### Add a Regression line with SciKit Learn

In [24]:
from sklearn.linear_model import LinearRegression
import numpy as np

train_x = msft3.index.value_counts().cumsum().values.reshape(-1,1)

#pandas['column].values -> converts column to numpy array

train_y = msft3['Close'].values.reshape(-1,1)

print(train_x[:5])

[[1]
 [2]
 [3]
 [4]
 [5]]


### Create a Linear Model

In [0]:
# create a linear model from the LinearRegression class
reg = LinearRegression()

### Train the model

In [26]:
# fit data to model
reg.fit(train_x, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Create predictions and assign them to a new column in msft3

In [0]:
msft3['LinReg'] = reg.predict(train_x)

#### Show the coefficient and the exponent for:  y = mx + b

In [28]:
print(reg.intercept_)
print(reg.coef_)

[57.90678566]
[[0.11581527]]


In [29]:
msft3.head(2)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,LinReg
Date,,,,,,,,,,,,,
2017-01-03,62.79,62.84,62.125,62.58,20694101.0,0.0,1.0,60.377440,60.425519,59.737991,60.175508,20694101.0,58.022601
2017-01-04,62.48,62.75,62.120,62.30,21339969.0,0.0,1.0,60.079351,60.338977,59.733183,59.906267,21339969.0,58.138416


### Graph it next to the Closing price data!

In [31]:
linreg_line = go.Scatter(
    x=msft3.index,
    y = msft3['LinReg'],
    name= 'Linear Regression',
    line = {'color' : 'blue','width' : 1 }
)

data = [ close_line, linreg_line]

# plot the data
configure_plotly_browser_state()
iplot(data)

### Create a 20 day moving average on the closing price values.  Whoo Rolling Averages!

In [32]:

msft3['20ma'] = msft3['Close'].rolling(20, min_periods=1).mean()

msft3.head(2)

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,LinReg,20ma
Date,,,,,,,,,,,,,,
2017-01-03,62.79,62.84,62.125,62.58,20694101.0,0.0,1.0,60.377440,60.425519,59.737991,60.175508,20694101.0,58.022601,62.58
2017-01-04,62.48,62.75,62.120,62.30,21339969.0,0.0,1.0,60.079351,60.338977,59.733183,59.906267,21339969.0,58.138416,62.44


### Plot the 20 day moving average

In [34]:
ma_line = go.Scatter(
    x=msft3.index,
    y = msft3['20ma'],
    name= '20 Day MA',
    line = {'color' : 'green','width' : 1 }
)

data = [ close_line, linreg_line, ma_line]


# plot the data
configure_plotly_browser_state()
iplot(data)

### Compute Bollinger Bands with the 20 day moving standard deviation and assign a lower and upper bound based on 20 Day MA +/- 2* 20 day MSD

In [0]:
# 20 day standard deviation

msft3['20sd'] = msft3['Close'].rolling(20, min_periods=1).std()

### Create the bands

In [0]:
# Upper band and lower band
msft3['Upper'] = msft3['20ma'] + 2 * msft3['20sd']

msft3['Lower'] = msft3['20ma'] - 2 * msft3['20sd']

### Graph the bans together with the stock price

In [38]:
Upper = go.Scatter(
    x=msft3.index,
    y = msft3['Upper'],
    name= 'Upper Band',
    fill = 'tonexty',
    line = {'color' : 'light green','width' : 1 }
)

Lower = go.Scatter(
    x=msft3.index,
    y = msft3['Lower'],
    name= 'Lower Band',
    fill = 'tonexty',
    line = {'color' : 'light green','width' : 1 }
)

data = [ close_line, Upper, Lower]

layout = go.Layout({
    'title' : 'Microsoft Stock with 20 Day MA Bollinger Bands',
    'yaxis' : {'title' : 'MSFT Stock ($)'},
    'xaxis' : {'rangeslider' : {'visible':False}}
})

fig = go.Figure(data=data,layout=layout)

# plot the data
configure_plotly_browser_state()
iplot(fig)

### Let's create a signal book for buying and selling Microsoft Stock using the Bollinger Bands as signals

In [39]:
msft4 = msft3[['Close','Upper','Lower']]
msft4.head(2)

,Close,Upper,Lower
Date,,,
2017-01-03,62.58,NaN,NaN
2017-01-04,62.30,62.83598,62.04402


#### If the closing price is below the lower band, that is a 'buy' signal and if the closing price is above the upper band, that is a 'sell' signal

In [40]:
# create a new column called Signal

msft4['Signal'] = np.where(msft4['Close'] > msft4['Upper'],'SELL','HOLD')
msft4['Signal'] = np.where(msft4['Close'] < msft4['Lower'],'BUY',msft4['Signal'])

msft4 = msft4[msft4['Signal'] != 'HOLD']

msft4 = msft4[msft4['Signal'] != msft4['Signal'].shift(1)]

msft4['Symbol'] = 'MSFT'
msft4['Shares'] = 1000

msft5 = msft4

msft5



,Close,Upper,Lower,Signal,Symbol,Shares
Date,,,,,,
2017-01-24,63.52,63.360715,62.016618,SELL,MSFT,1000
2017-08-10,71.41,74.549690,71.528310,BUY,MSFT,1000
2017-08-30,74.01,73.935255,71.582745,SELL,MSFT,1000
2017-12-04,81.08,85.387375,81.816625,BUY,MSFT,1000
2017-12-15,86.85,86.630036,80.903964,SELL,MSFT,1000
2018-02-08,85.01,95.984196,85.980804,BUY,MSFT,1000
2018-04-17,96.07,95.989556,87.217444,SELL,MSFT,1000
2018-06-27,97.54,103.254016,98.085984,BUY,MSFT,1000
2018-07-12,104.19,103.676964,97.229036,SELL,MSFT,1000


### Excellent!!!  We just created some trading signals by building out technical indicators and visualizing them using the Quandl API, Request web library, and the Plotly graphing library.  Please return to the original jupyter notebook file to continue on with the lesson!